In [1]:
import numpy as np
import pandas

from asodesigner.consts import *
from data_genertion.consts import *
from asodesigner.util import get_antisense, get_gc_content

In [2]:
all_data = pandas.read_csv(DATA_PATH / 'data_from_article_fixed.csv')

In [3]:
all_data['gc_content'] = [get_gc_content(seq) for seq in all_data[SEQUENCE]]

In [4]:
from scipy.stats import pearsonr, spearmanr

def print_correlations(df, name1, name2, p_value_threshold=None):
    if p_value_threshold is None:
        p_value_threshold = 1
    corr, p_value = pearsonr(df[name1], df[name2])
    if p_value < p_value_threshold:
        print(f"Feature: {name1:<35}, Pearson: {corr:<5.2f}, p-value: {p_value:<10.2} Target: {name2:<35}")
    corr, p_value = spearmanr(df[name1], df[name2])
    if p_value < p_value_threshold:
        print(f"Feature: {name1:<35}, Spearman: {corr:<5.2f}, p-value: {p_value:<10.2} Target: {name2:<35}")


In [5]:
all_data_no_nan = all_data.dropna(subset=[INHIBITION]).copy()

In [6]:
all_data_no_nan.loc[:, 'log_inhibition'] = -np.log(-all_data_no_nan[INHIBITION] + 100.001)

In [7]:
import ViennaRNA as RNA
all_data_no_nan.loc[:, 'internal_fold'] = [RNA.fold(antisense)[1] for antisense in all_data_no_nan[SEQUENCE]]

print_correlations(all_data_no_nan, 'internal_fold', INHIBITION)
print_correlations(all_data_no_nan, 'internal_fold', 'log_inhibition')

Feature: internal_fold                      , Pearson: 0.01 , p-value: 0.038      Target: Inhibition(%)                      
Feature: internal_fold                      , Spearman: 0.00 , p-value: 0.7        Target: Inhibition(%)                      
Feature: internal_fold                      , Pearson: 0.03 , p-value: 1.7e-06    Target: log_inhibition                     
Feature: internal_fold                      , Spearman: 0.00 , p-value: 0.7        Target: log_inhibition                     


In [8]:
CANONICAL_GENE = 'Canonical Gene Name'
CELL_LINE_ORGANISM = 'Cell line organism'

all_data_no_nan_human = all_data_no_nan[all_data_no_nan[CELL_LINE_ORGANISM] == 'human']
genes = all_data_no_nan[CANONICAL_GENE].copy()
genes_u = list(set(genes))
genes_u.remove('HBV')
genes_u.remove('negative_control')

In [9]:
from asodesigner.read_human_genome import get_locus_to_data_dict
import pickle
from asodesigner.consts import CACHE_DIR


cache_path = CACHE_DIR / 'gene_to_data_simple_cache.pickle'
if not cache_path.exists():
    gene_to_data = get_locus_to_data_dict(include_introns=True, gene_subset=genes_u)
    with open(cache_path, 'wb') as f:
        pickle.dump(gene_to_data, f)
else:
    with open(cache_path, 'rb') as f:
        gene_to_data = pickle.load(f)

In [10]:
for gene_name in genes_u:
    if gene_name not in gene_to_data:
        print(f"gene : {gene_name} not found")

In [12]:
from asodesigner.util import get_antisense

all_data_human_gene = all_data_no_nan_human[all_data_no_nan_human[CANONICAL_GENE].isin(genes_u)].copy()
PREMRNA_FOUND = 'pre_mrna_found'
SENSE_START = 'sense_start'
SENSE_LENGTH = 'sense_length'
SENSE_TYPE = 'sense_type'

found = 0
not_found = 0
all_data_human_gene[SENSE_START] = np.zeros_like(all_data_human_gene[CANONICAL_GENE], dtype=int)
all_data_human_gene[SENSE_LENGTH] = np.zeros_like(all_data_human_gene[CANONICAL_GENE], dtype=int)
all_data_human_gene[SENSE_TYPE] = "NA"
for index, row in all_data_human_gene.iterrows():
     gene_name = row[CANONICAL_GENE]
     locus_info = gene_to_data[gene_name]
     pre_mrna = locus_info.full_mrna
     antisense = row[SEQUENCE]
     sense = get_antisense(antisense)
     idx = pre_mrna.find(sense)
     all_data_human_gene.loc[index, SENSE_START] = idx
     all_data_human_gene.loc[index, SENSE_LENGTH] = len(antisense)
     if idx != -1:
         genome_corrected_index = idx + locus_info.exon_indices[0][0]
         found = False
         for exon_indices in locus_info.exon_indices:
            # print(exon[0], exon[1])
            if exon_indices[0] <=  genome_corrected_index <= exon_indices[1]:
                all_data_human_gene.loc[index, SENSE_TYPE] = 'exon'
                found = True
                break
     if not found:
         all_data_human_gene.loc[index, SENSE_TYPE] = 'intron'


In [13]:
all_data_human_gene_premrna = all_data_human_gene[all_data_human_gene[SENSE_START] != -1]
len(all_data_human_gene_premrna)


29987

In [ ]:
from asodesigner.fold import get_trigger_mfe_scores_by_risearch, get_mfe_scores, dump_target_file,Interaction
from asodesigner.modified_dsm import make_dsm_ps_dna_rna , make_dsm_dna_rna
from external.risearch.RIsearch1.numpy_to_csv import dsm_variable_to_csv,numpy_to_csv
import random

# minimal_scores = [200,400,600,800,1000,1200]
minimal_scores = [800]
on_target_feature_titles = ['on_target_energy_fits', 'on_target_energy_sum', 'on_target_energy_max']

all_data_human_gene_premrna = all_data_human_gene_premrna.copy() # so Jupyter doesn't shout at me

# genes_u.remove('HIF1A') # the same sequence for all ASOs, only differing chemical modifications
for score in minimal_scores:
    for feature_title in on_target_feature_titles:
        all_data_human_gene_premrna.loc[:, feature_title + str(score)] = np.zeros_like(all_data_human_gene_premrna[SEQUENCE], dtype=np.float64)

    for gene in genes_u:
        target = gene_to_data[gene].full_mrna
        name_to_sequence = {'target_seq' : target}
        results = []
        parsing_type = '2'

        hash = random.getrandbits(64)
        target_cache_filename = f'target-cache-{hash}.fa'
        target_cache_path = dump_target_file(target_cache_filename, name_to_sequence)

        gene_rows = all_data_human_gene_premrna[all_data_human_gene_premrna[CANONICAL_GENE] == gene]
        loop_args = [(Interaction.DNA_RNA_NO_WOBBLE,None),(Interaction.MODIFIED,make_dsm_dna_rna),(Interaction.MODIFIED,make_dsm_ps_dna_rna)]
        for args in loop_args:
            interaction , func = args
            if func:
                func()
            else:
                print(interaction)
                continue
            for index, row in gene_rows.iterrows():
                antisense = row[SEQUENCE]
                sense = get_antisense(antisense)
                l = row[SENSE_LENGTH]
                # TODO: din, add DNA-RNA, PSDNA-RNA
                tmp_results = get_trigger_mfe_scores_by_risearch(sense, name_to_sequence, interaction_type = interaction ,minimum_score=score, neighborhood=l,
                                                         parsing_type=parsing_type, target_file_cache=target_cache_path)
                scores = get_mfe_scores(tmp_results, parsing_type)
                if len(scores) == 0:
                    all_data_human_gene_premrna.loc[index, 'on_target_energy_fits' + str(score)] = 0.
                    all_data_human_gene_premrna.loc[index, 'on_target_energy_sum' + str(score)] = 0.
                    all_data_human_gene_premrna.loc[index, 'on_target_energy_max' + str(score)] = 0.
                else:
                    target_scores = scores[0]
                    min_score = 0 if len(target_scores) == 0 else min(target_scores)
                    all_data_human_gene_premrna.loc[index, 'on_target_energy_fits' + str(score)] = len(target_scores)
                    all_data_human_gene_premrna.loc[index, 'on_target_energy_sum' + str(score)] = sum(target_scores)
                    all_data_human_gene_premrna.loc[index, 'on_target_energy_max' + str(score)] = min_score



Interaction.DNA_RNA_NO_WOBBLE
['trigger', '1', '16', 'target_seq', '17617', '17632', '1119', '-18.30']
['trigger', '1', '16', 'target_seq', '18957', '18972', '1184', '-18.95']
['trigger', '11', '16', 'target_seq', '193', '198', '804', '-9.15']
['trigger', '9', '16', 'target_seq', '866', '873', '899', '-11.30']
['trigger', '11', '16', 'target_seq', '1407', '1412', '804', '-9.15']
['trigger', '10', '16', 'target_seq', '1559', '1565', '844', '-10.15']
['trigger', '10', '16', 'target_seq', '3128', '3134', '844', '-10.15']
['trigger', '11', '16', 'target_seq', '3716', '3721', '804', '-9.15']
['trigger', '8', '15', 'target_seq', '3778', '3785', '874', '-11.05']
['trigger', '11', '16', 'target_seq', '4112', '4117', '804', '-9.15']
['trigger', '11', '16', 'target_seq', '7203', '7208', '804', '-9.15']
['trigger', '11', '16', 'target_seq', '9785', '9790', '804', '-9.15']
['trigger', '11', '16', 'target_seq', '9857', '9862', '804', '-9.15']
['trigger', '11', '16', 'target_seq', '11599', '11604', 

In [ ]:
on_target_energy_features = ['on_target_energy_fits' + str(score) for score in minimal_scores] + ['on_target_energy_sum' + str(score) for score in minimal_scores] + ['on_target_energy_max' + str(score) for score in minimal_scores]
on_target_energy_features

In [ ]:
for score in minimal_scores:
    for on_target_name in on_target_feature_titles:
        all_data_human_gene_premrna.loc[:, on_target_name + "_normalized" + str(score)] = all_data_human_gene_premrna['on_target_energy_fits' + str(score)] / all_data_human_gene_premrna[SENSE_LENGTH]
        on_target_energy_features.append(on_target_name + "_normalized" + str(score))

In [ ]:
import matplotlib.pyplot as plt

test = all_data_human_gene_premrna[all_data_human_gene_premrna[CANONICAL_GENE] == 'SNCA']
test = test.sort_values(by=['on_target_energy_max800'], ascending=[False])
print(test.tail(5))
plt.scatter(test['on_target_energy_max800'], test['log_inhibition'], alpha=0.05)
plt.show()

In [ ]:
from fold import calculate_energies, get_weighted_energy

all_data_human_gene_premrna_no_nan = all_data_human_gene_premrna.copy()

# Comment out the long cases for quick running
# fold_variants = [(40, 15), (60, 15), (80, 15), (100, 15), (60, 25), (80, 25), (100, 25), (80, 40), (100, 40)]
fold_variants = [(40, 15)]
for (window_size, step_size) in fold_variants:
    for gene in genes_u:
        on_target_fold = 'on_target_fold_openness' + str(window_size) + '_' + str(step_size)
        on_target_fold_normalized = 'on_target_fold_openness_normalized' + str(window_size) + '_' + str(step_size)
        # on_target_fold = 'on_target_fold_openness'
        # on_target_fold_normalized = 'on_target_fold_openness_normalized'


        all_data_human_gene_premrna_no_nan[on_target_fold] = np.zeros_like(all_data_human_gene_premrna_no_nan[SEQUENCE], dtype=np.float64)
        all_data_human_gene_premrna_no_nan[on_target_fold_normalized] = np.zeros_like(all_data_human_gene_premrna_no_nan[SEQUENCE], dtype=np.float64)


        target = gene_to_data[gene].full_mrna
        gene_rows = all_data_human_gene_premrna_no_nan[all_data_human_gene_premrna_no_nan[CANONICAL_GENE] == gene]
        energies = calculate_energies(str(target), step_size, window_size)

        for index, row in gene_rows.iterrows():
            antisense = row[SEQUENCE]
            sense = get_antisense(antisense)
            l = row[SENSE_LENGTH]
            sense_start = row[SENSE_START]
            mean_fold = get_weighted_energy(sense_start, l, step_size, energies, window_size)

            all_data_human_gene_premrna_no_nan.loc[index, on_target_fold] = mean_fold
            all_data_human_gene_premrna_no_nan.loc[index, on_target_fold_normalized] = mean_fold / l


In [ ]:
fold_feature_names = ['on_target_fold_openness', 'on_target_fold_openness_normalized']
fold_features = []
for (window_size, step_size) in fold_variants:
    for feature_name in fold_feature_names:
        fold_features.append(feature_name + str(window_size) + "_" + str(step_size))

In [ ]:
print(all_data_human_gene_premrna_no_nan.columns)

In [ ]:
test = all_data_human_gene_premrna_no_nan
# test = all_data_human_gene_premrna_no_nan[all_data_human_gene_premrna_no_nan[INHIBITION] > 0]


for feature_name in fold_features:
    print_correlations(test , feature_name, INHIBITION, p_value_threshold=0.05)
    print_correlations(test , feature_name, 'log_inhibition', p_value_threshold=0.05)

feature = fold_features[0]
plt.scatter(test[feature], test[INHIBITION], alpha=0.05)
plt.show()


In [ ]:
all_data_human_gene_premrna_no_nan.loc[:, 'energy_and_fold'] = all_data_human_gene_premrna_no_nan['on_target_energy_max800'] * all_data_human_gene_premrna_no_nan['on_target_fold_openness_normalized40_15']

print_correlations(all_data_human_gene_premrna_no_nan, 'energy_and_fold', INHIBITION)
print_correlations(all_data_human_gene_premrna_no_nan, 'energy_and_fold', 'log_inhibition')

In [ ]:
import primer3

all_data_human_gene_premrna_no_nan.loc[:, 'self_energy'] = [float(primer3.calc_homodimer(antisense).dg) for antisense in all_data_human_gene_premrna_no_nan[SEQUENCE]]
all_data_human_gene_premrna_no_nan.loc[:, 'self_energy'] = all_data_human_gene_premrna_no_nan.loc[:, 'self_energy'].astype(float)

print_correlations(all_data_human_gene_premrna_no_nan, 'self_energy', INHIBITION)
print_correlations(all_data_human_gene_premrna_no_nan, 'self_energy', 'log_inhibition')

In [ ]:
all_data_human_gene_premrna_no_nan.loc[:, 'internal_fold'] = [RNA.fold(antisense)[1] for antisense in all_data_human_gene_premrna_no_nan[SEQUENCE]]

print_correlations(all_data_human_gene_premrna_no_nan, 'internal_fold', INHIBITION)
print_correlations(all_data_human_gene_premrna_no_nan, 'internal_fold', 'log_inhibition')

In [ ]:
all_data_human_gene_premrna_no_nan.loc[:, 'gc_content_5_prime_5'] = [get_gc_content(sequence[:5]) for sequence in all_data_human_gene_premrna_no_nan[SEQUENCE]]
all_data_human_gene_premrna_no_nan.loc[:, 'gc_content_3_prime_5'] = [get_gc_content(sequence[-5:]) for sequence in all_data_human_gene_premrna_no_nan[SEQUENCE]]
all_data_human_gene_premrna_no_nan.loc[:, 'first_nucleotide'] = [sequence[0] for sequence in all_data_human_gene_premrna_no_nan[SEQUENCE]]
all_data_human_gene_premrna_no_nan.loc[:, 'second_nucleotide'] = [sequence[2] for sequence in all_data_human_gene_premrna_no_nan[SEQUENCE]]

print_correlations(all_data_human_gene_premrna_no_nan, 'gc_content_5_prime_5', INHIBITION)
print_correlations(all_data_human_gene_premrna_no_nan, 'gc_content_5_prime_5', 'log_inhibition')
print_correlations(all_data_human_gene_premrna_no_nan, 'gc_content_3_prime_5', 'log_inhibition')
print_correlations(all_data_human_gene_premrna_no_nan, 'gc_content_3_prime_5', 'log_inhibition')

In [ ]:
import primer3

all_data_human_gene_premrna_no_nan.loc[:, 'self_energy'] = [primer3.calc_homodimer(antisense).dg for antisense in all_data_human_gene_premrna_no_nan[SEQUENCE]]
all_data_human_gene_premrna_no_nan.loc[:, 'self_energy_log'] = -np.log(2500-all_data_human_gene_premrna_no_nan.loc[:, 'self_energy'])

print_correlations(all_data_human_gene_premrna_no_nan, 'self_energy', INHIBITION)
print_correlations(all_data_human_gene_premrna_no_nan, 'self_energy', 'log_inhibition')
print_correlations(all_data_human_gene_premrna_no_nan, 'self_energy_log', INHIBITION)
print_correlations(all_data_human_gene_premrna_no_nan, 'self_energy_log', 'log_inhibition')

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(all_data_human_gene_premrna_no_nan['self_energy_log'], all_data_human_gene_premrna_no_nan['log_inhibition'], alpha=0.1)
plt.show()

In [ ]:
print_correlations(all_data_human_gene_premrna_no_nan, TREATMENT_PERIOD, INHIBITION)
print_correlations(all_data_human_gene_premrna_no_nan, TREATMENT_PERIOD, 'log_inhibition')
plt.scatter(all_data_human_gene_premrna_no_nan[TREATMENT_PERIOD], all_data_human_gene_premrna_no_nan[INHIBITION], alpha=0.05)
plt.show()

In [ ]:
tmp = all_data_human_gene_premrna_no_nan.copy()
tmp = tmp.dropna(subset=[DENSITY]).copy()
tmp.loc[:, 'log_density'] = np.log(tmp[DENSITY])

print_correlations(tmp, DENSITY, INHIBITION)
print_correlations(tmp, DENSITY, 'log_inhibition')
print_correlations(tmp, 'log_density', INHIBITION)
print_correlations(tmp, 'log_density', 'log_inhibition')
plt.scatter(tmp[DENSITY], tmp[INHIBITION], alpha=0.05)
plt.show()

In [ ]:
all_data_human_gene_premrna_no_nan.loc[:, 'mrna_length'] = [len(gene_to_data[gene].full_mrna) for gene in all_data_human_gene_premrna_no_nan[CANONICAL_GENE]]
all_data_human_gene_premrna_no_nan.loc[:, 'normalized_start'] = all_data_human_gene_premrna_no_nan[SENSE_START] / all_data_human_gene_premrna_no_nan['mrna_length']

print_correlations(all_data_human_gene_premrna_no_nan, 'normalized_start', INHIBITION)
print_correlations(all_data_human_gene_premrna_no_nan, 'normalized_start', 'log_inhibition')


plt.scatter(all_data_human_gene_premrna_no_nan['normalized_start'], all_data_human_gene_premrna_no_nan['log_inhibition'], alpha=0.05)
plt.show()

In [ ]:
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection  import train_test_split, cross_val_score
from sklearn.metrics import make_scorer
import pandas as pd

filtered = all_data_human_gene_premrna_no_nan.copy()
filtered = filtered[filtered['Cell line organism'] == 'human']
print(len(filtered))


filtered = filtered.dropna(subset=[INHIBITION]).copy()
filtered = filtered.dropna(subset=[DENSITY]).copy()


filtered['log_volume'] = np.log(filtered[VOLUME])
filtered.loc[:, 'log_inhibition'] = -np.log(-filtered[INHIBITION] + 101)
filtered.loc[:, 'log_density'] = np.log(filtered[DENSITY])
filtered.loc[:, SENSE_LENGTH] = [len(sequence) for sequence in filtered[SEQUENCE]]
filtered.loc[:, SENSE_LENGTH] = filtered.loc[:, SENSE_LENGTH].astype(float)
filtered['gc_deviation'] = np.abs(0.5 - filtered['gc_content'])
filtered['self_energy_log_squared'] = filtered['self_energy_log'] ** 2
# filtered['self_energy_squared'] = filtered['self_energy'] ** 2

ONE_HOT_FEATURE_NAMES = [CELL_LINE, DENSITY, TRANSFECTION, 'first_nucleotide', 'second_nucleotide', MODIFICATION, SENSE_TYPE]
for feature in ONE_HOT_FEATURE_NAMES:
    filtered = pd.get_dummies(filtered, columns=[feature]).copy()

# features_base = ['log_volume', TREATMENT_PERIOD, VOLUME]
features_base = ['gc_content', 'gc_deviation', SENSE_LENGTH, 'normalized_start', SENSE_START, 'self_energy_log', 'self_energy_log_squared', 'internal_fold', 'energy_and_fold', 'gc_content_3_prime_5', 'gc_content_5_prime_5']
experiment_features = [VOLUME, 'log_volume', TREATMENT_PERIOD, 'log_density']

one_hot_encoded_features = []
for one_hot_feature in ONE_HOT_FEATURE_NAMES:
    one_hot_encoded_features.append([feature for feature in filtered.columns if one_hot_feature in feature and one_hot_feature != feature])
# Flatten the list of list to a single big list
flat_one_hot_encoded_features = [feature for sublist in one_hot_encoded_features for feature in sublist]

features = features_base + experiment_features + on_target_energy_features + fold_features + flat_one_hot_encoded_features

print("Used features: ", features)

# filtered = all_data_human_gene_premrna_no_nan
common_chemical_pattern = 'CCCddddddddddCCC'

# filtered = filtered[filtered['Chemical_Pattern'] == common_chemical_pattern]
# filtered = filtered[filtered[CANONICAL_GENE] == 'IRF5']
# filtered = filtered[filtered[INHIBITION] > 0]
# filtered = filtered[filtered[VOLUME] == 4000]
# filtered = filtered[filtered[INHIBITION] > 70]

# train, test = train_test_split(filtered, test_size=0.2, random_state=0, stratify=filtered[CANONICAL_GENE])
train, test = train_test_split(filtered, test_size=0.2)

print("Test: ", len(test))
print("Train: ", len(train))
X = train[features]
y = train['log_inhibition']
# x_tag = X
# y_tag = y

weight = VOLUME


# Multiply the values via numpy without changing the column names
multiplier = np.sqrt(X[weight]).values.reshape(-1, 1)
X_tag = pd.DataFrame(
    X[features].values * multiplier,
    index=X.index,
    columns=X[features].columns)


y_tag  = y * np.sqrt(X[weight])

model = LinearRegression()
# model.fit(X_tag, y_tag)
# model = RandomForestRegressor(n_jobs=-1)

# model = RandomForestRegressor(random_state=42, n_jobs=-1)

# model.fit(X,y)
model.fit(train[features], train['log_inhibition'])


baseline_score = cross_val_score(model, X, y, cv=5).mean()
print("Baseline score: ", baseline_score)

def spearman_score(y_true, y_pred):
    return stats.spearmanr(y_true, y_pred).correlation ** 2

spearman_scorer = make_scorer(spearman_score, greater_is_better=True)
spearman_score = cross_val_score(model, X, y, cv=5, scoring=spearman_scorer, n_jobs=-1).mean()
print("Spearman score: ", spearman_score)



In [ ]:
test_filtered = test.copy()
# test_filtered = test_filtered[test_filtered[CANONICAL_GENE] == 'MAPT']
# test_filtered = test_filtered[test_filtered['Chemical_Pattern'] == 'CCCddddddddddCCC']
# test_filtered = test_filtered[test_filtered[INHIBITION] > 90]

test_features = test_filtered[features]
#
# multiplier = np.sqrt(test[weight]).values.reshape(-1, 1)
# test_features = pd.DataFrame(
#     test[features].values * multiplier,
#     index=test.index,
#     columns=test[features].columns)
predicted = model.predict(test_filtered[features])


# test_features = test_filtered[features]
# predicted = model.predict(test_features)
# y_test = test_filtered['log_inhibition'] / np.sqrt(test_filtered[weight])
y_test = test_filtered['log_inhibition']


corr, p_value = pearsonr(y_test, predicted)
print("Pearson Corr: ", corr, " p_value: ", p_value)
corr, p_value = spearmanr(y_test, predicted)
print("Spearman Corr: ", corr, " p_value: ", p_value)
plt.scatter(predicted, y_test, alpha=0.2)
# plt.scatter(y_test, test['gc_content'])
x = np.linspace(np.min(predicted), np.max(predicted), 100)
plt.plot(x, x, color='red')

plt.xlabel('Predicted Inhibition')
plt.ylabel('Actual Inhibition')
plt.title('Predicted vs Actual Inhibition')
plt.show()

In [ ]:
from sklearn.model_selection import permutation_test_score

score, permutation_scores, pvalue = permutation_test_score(model, X, y, cv=5, n_permutations=100, n_jobs=-1, scoring='r2')
print(score, permutation_scores, pvalue)

In [ ]:
plt.scatter(filtered['self_energy_log_squared'], filtered['log_inhibition'], alpha=0.1)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import rankdata

def partial_corr(x, y, z):
    # Regress x on z
    X = sm.add_constant(z)
    model_x = sm.OLS(x, X).fit()
    res_x = model_x.resid

    # Regress y on z
    model_y = sm.OLS(y, X).fit()
    res_y = model_y.resid

    # Compute correlation between the residuals
    return np.corrcoef(res_x, res_y)[0, 1]

def partial_spearman_corr(x, y, z):
    # Convert variables to ranks
    rx = rankdata(x)
    ry = rankdata(y)
    rz = rankdata(z)

    # Regress rx on rz
    X = sm.add_constant(rz)
    model_x = sm.OLS(rx, X).fit()
    res_x = model_x.resid

    # Regress ry on rz
    model_y = sm.OLS(ry, X).fit()
    res_y = model_y.resid

    # Compute the Pearson correlation between the residuals
    return np.corrcoef(res_x, res_y)[0, 1]


for gene in genes_u:
    t = filtered[filtered[CANONICAL_GENE] == gene]
    print("Gene:", gene)
    try:
        pcorr = partial_spearman_corr(t['self_energy'], t['log_inhibition'], t['gc_content'])
        print(pcorr)
    except:
        pass

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(filtered[features], filtered['log_inhibition'])

# Get the feature importances
importances = rf.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)



In [ ]:
for index, row in importance_df.iterrows():
    print(f"{row['Feature']:<40} {row['Importance']:.3}")


In [ ]:
print("The baseline is: ", baseline_score)
print("The spearman baseline is: ", spearman_score)
cv = 5
scorer = spearman_scorer
jobs = -1
feature_importances = []

# for feature in features_base + on_target_energy_features + fold_features:
for feature in features_base:
    score_without_feature = cross_val_score(model, X.drop(columns=[feature]), y, cv=cv, scoring=scorer, n_jobs=jobs).mean()
    # print(f"Dropping {feature}: performance drop = {baseline_score - score_without_feature}")
    feature_importances.append((feature, float(baseline_score - score_without_feature)))

print("Done base features")

for i in range(len(one_hot_encoded_features)):
    feature_name = ONE_HOT_FEATURE_NAMES[i]
    feature = one_hot_encoded_features[i]

    score_without_feature = cross_val_score(model, X.drop(columns=feature), y, cv=cv, scoring=scorer, n_jobs=jobs).mean()
    performance_diff = baseline_score - score_without_feature
    feature_importances.append((feature_name, float(performance_diff)))


score_without_on_target_energy = cross_val_score(model, X.drop(columns=on_target_energy_features), y, cv=cv, scoring=scorer, n_jobs=jobs).mean()
performance_drop_all_on_target_energy = baseline_score - score_without_on_target_energy
feature_importances.append(("All on target energy", float(performance_drop_all_on_target_energy)))

score_without_fold_features = cross_val_score(model, X.drop(columns=fold_features), y, cv=cv, scoring=scorer, n_jobs=jobs).mean()
performance_drop_fold_features = baseline_score - score_without_fold_features
feature_importances.append(("Fold features", float(performance_drop_fold_features)))


sorted_data = sorted(feature_importances, key=lambda x: x[1], reverse=True)
for datum in sorted_data:
    print(datum)


In [ ]:
print(filtered.columns)


for gene in genes_u:
    print("Gene:", gene)

    try:
        predicted_gene = model.predict(filtered[filtered[CANONICAL_GENE] == gene][features])
        y_gene = filtered[filtered[CANONICAL_GENE] == gene]['log_inhibition']
        corr, p_value = pearsonr(y_gene, predicted_gene)
        print("Pearson Corr: ", corr, " p_value: ", p_value)
        corr, p_value = spearmanr(y_gene, predicted_gene)
        print("Spearman Corr: ", corr, " p_value: ", p_value)
    except:
        pass
# plt.scatter(predicted, y)
# plt.show()

In [ ]:
for gene in genes_u:
    print("Gene: ", gene)
    print(len(set(list(all_data_human_gene_premrna_no_nan[all_data_human_gene_premrna_no_nan[CANONICAL_GENE] == gene][SEQUENCE]))))

In [ ]:
subset = ['Sequence', 'Chemical_Pattern', 'Smiles', 'Linkage', 'Location', 'Modification', 'Primer_probe_set', 'ISIS', 'Cell_line', VOLUME, DENSITY]
result = all_data_human_gene_premrna_no_nan.groupby(subset).filter(lambda x: len(x) > 20)
# result = all_data_human_gene_premrna_no_nan
result = result.sort_values(by=['Sequence'])
result